1. Get the bind 
2. Invoke the prediction for the corner of the bins 

In [1]:
import numpy as np 
from itertools import product
import pandas as pd

df = pd.read_pickle('detrended.pkl')

In [2]:
def _get_quantiles(train_set, feature, bins):
    """Get quantiles from a feature in a dataset.
    Parameters
    ----------
    train_set : pandas.core.frame.DataFrame
        Dataset containing feature `feature`.
    feature : column label
        Feature for which to calculate quantiles.
    bins : int
        The number of quantiles is calculated as `bins + 1`.
    Returns
    -------
    quantiles : array-like
        Quantiles.
    bins : int
        Number of bins, `len(quantiles) - 1`. This may be lower than the original
        `bins` if identical quantiles were present.
    Raises
    ------
    ValueError
        If `bins` is not an integer.
    Notes
    -----
    When using this definition of quantiles in combination with a half open interval
    (lower quantile, upper quantile], care has to taken that the smallest observation
    is included in the first bin. This is handled transparently by `np.digitize`.
    """
    if not isinstance(bins, (int, np.integer)):
        raise ValueError(
            "Expected integer 'bins', but got type '{}'.".format(type(bins))
        )
    quantiles = np.unique(
        np.quantile(
            train_set[feature], np.linspace(0, 1, bins + 1), interpolation="lower"
        )
    )
    bins = len(quantiles) - 1
    return quantiles, bins

In [3]:
_get_quantiles(df, 'TI-19', 10)

(array([-11.23479291,  -2.53481609,  -1.53466239,  -0.98140235,
         -0.64641399,  -0.20998329,   0.41364643,   1.06293961,
          1.55717116,   1.98567802,  16.37713454]),
 10)

In [4]:
train_set = df

In [5]:
quantiles_list, bins_list = tuple(
    zip(
        *(
            _get_quantiles(train_set, feature, n_bin)
            for feature, n_bin in zip(['TI-19', 'FI-2'], [10,10])
        )
    )
)

# Define the bins the feature samples fall into. Shift and clip to ensure we are
# getting the index of the left bin edge and the smallest sample retains its index
# of 0.
indices_list = [
    np.clip(np.digitize(train_set[feature], quantiles, right=True) - 1, 0, None)
    for feature, quantiles in zip(['TI-19', 'FI-2'], quantiles_list)
]

In [6]:
indices_list[0]

array([2, 2, 2, ..., 1, 2, 2])

In [7]:
quantiles_list[0]

array([-11.23479291,  -2.53481609,  -1.53466239,  -0.98140235,
        -0.64641399,  -0.20998329,   0.41364643,   1.06293961,
         1.55717116,   1.98567802,  16.37713454])

In [8]:
features = ['TI-19', 'FI-2']

In [17]:
predictions = {}
for shifts in product(*(range(2),) * 2):
    print(shifts)
    mod_train_set = train_set.copy()
    for i in range(2):
        mod_train_set[features[i]] = quantiles_list[i][indices_list[i] + shifts[i]]
        #print(mod_train_set)
#     predictions[shifts] = predictor(mod_train_set)
# # The individual effects.
# effects = (predictions[(1, 1)] - predictions[(1, 0)]) - (
#     predictions[(0, 1)] - predictions[(0, 0)]
# )

(0, 0)
(0, 1)
(1, 0)
(1, 1)


In [10]:
quantiles_list[0][indices_list[0] + shifts[0]]

array([-0.98140235, -0.98140235, -0.98140235, ..., -1.53466239,
       -0.98140235, -0.98140235])

In [11]:
indices_list[0]

array([2, 2, 2, ..., 1, 2, 2])

In [12]:
quantiles_list[0]

array([-11.23479291,  -2.53481609,  -1.53466239,  -0.98140235,
        -0.64641399,  -0.20998329,   0.41364643,   1.06293961,
         1.55717116,   1.98567802,  16.37713454])

In [20]:
shifts

(1, 1)

In [22]:
quantiles_list[i][indices_list[i] + shifts[i]]

12609

In [19]:
mod_train_set['FI-19']

2020-01-15 11:42:01    25.022335
2020-01-15 11:44:01    27.723213
2020-01-15 11:46:01    38.114764
2020-01-15 11:48:01    40.775602
2020-01-15 11:50:01    11.225991
                         ...    
2020-02-01 23:50:01    34.206385
2020-02-01 23:52:01    16.645300
2020-02-01 23:54:01    27.047838
2020-02-01 23:56:01    20.176938
2020-02-01 23:58:01    30.267708
Freq: 2T, Name: FI-19, Length: 12609, dtype: float64

In [27]:
len(np.where((df['TI-19']>=-11.23479291) & (df['TI-19'] < -2.53481609))[0])

1260